In [41]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import numpy as np
from sklearn.mixture import GaussianMixture
from Exam.PB2.soft_clustering_measure import v_measure


BASE_DIR = '/Users/aman/Dropbox/CS6220_Amandeep_Singh/Exam/PB2'
DATA_DIR = '{0}/sample_dataset1.txt'.format(BASE_DIR)
N_FEATURES = 1000
N_TOP_WORDS = 20
T = 20
K = 20

In [6]:
# with open('/Users/aman/Dropbox/CS6220_Amandeep_Singh/Exam/PB2/whole_dataset.txt', 'r') as file:
#     data = file.readlines()
#     id = []
#     topic = []
#     text = []
#     for line in data:
#         info = line.split(',')
#         id.append(info[0])
#         topic.append(info[1][1:-1])
#         text.append(''.join(info[2:]).replace('"', ''))
#     d = {'id': id, 'topic': topic, 'text': text}
#     df = pd.DataFrame(data=d).to_csv(BASE_DIR + '/whole.csv')

# with open('/Users/aman/Dropbox/CS6220_Amandeep_Singh/Exam/PB2/whole_dataset.txt', 'r') as file:
#     data = file.readlines()
#     id = []
#     topic = []
#     text = []
#     for line in data:
#         info = line.split(',')
#         id.append(info[0])
#         topic.append(info[1][2:-1])
#         text.append(''.join(info[2:]).replace('"', ''))
#     d = {'id': id, 'topic': topic, 'text': text}
#     df = pd.DataFrame(data=d).to_csv(BASE_DIR + '/whole.csv')

In [8]:
whole_data = pd.read_csv(BASE_DIR + '/whole.csv', names=['id', 'text', 'topic'], skiprows=1)
sample_data = pd.read_csv(BASE_DIR + '/new.csv', names=['id', 'text', 'topic'], skiprows=1)

In [9]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,max_features=N_FEATURES,stop_words='english')
tf = tf_vectorizer.fit_transform(whole_data.text.tolist()).todense()


In [17]:
lda = LatentDirichletAllocation(n_components=T, learning_method='online')
lda_transform = lda.fit_transform(tf)

In [20]:
gm = GaussianMixture(n_components=K)

In [21]:
gm.fit(lda_transform[sample_data.id.tolist(),:])

GaussianMixture(covariance_type='full', init_params='kmeans', max_iter=100,
        means_init=None, n_components=20, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [43]:
# predictions = gm.predict(lda_transform[sample_data.id.tolist(),:])
label_true = [np.argmax(lda.transform(i)[0]) for i in tf[sample_data.id.tolist()]] 
labels_pred_prob = gm.predict_proba(lda_transform[sample_data.id.tolist(), :])
print(v_measure(labels_pred_prob,label_true,20,20))

(5.126711544023337, -inf, nan)


/Users/aman/.envs/digit_rec/lib/python2.7/site-packages/scipy/stats/_distn_infrastructure.py:2508: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=0)
/Users/aman/Dropbox/CS6220_Amandeep_Singh/Exam/PB2/soft_clustering_measure.py:50: RuntimeWarning: invalid value encountered in double_scalars
  (homogeneity + completeness))


In [45]:
np.argpartition((np.sum(gm.predict_proba(lda_transform[sample_data.id.tolist(), :]),axis=0)),-5)[-5:]

array([17, 14,  7,  6, 12])

In [53]:
n_top_words = 20
feature_names = tf_vectorizer.get_feature_names()
# def print_top_words(lda_transform, feature_names, n_top_words):
for topic_idx, topic in enumerate(lda.components_[[17, 14,  7,  6, 12]]):
    message = "Topic #%d: " % topic_idx
    index = topic.argsort()[:-n_top_words - 1:-1]
    message += " ".join([feature_names[i] + " " + str(topic[i] / topic.sum()) for i in index])
    print(message)

Topic #0: gun 0.0498639980749774 right 0.04517926815549314 rights 0.03704282363250262 games 0.03392671422586689 war 0.03376705520620853 country 0.027161360417462276 military 0.025623780609136393 killed 0.025468446532813365 involved 0.02184275761483738 peace 0.02068653360574068 win 0.02016428045177756 control 0.019340171970219148 red 0.01903214449666049 sex 0.018000629202713427 force 0.017023798346569176 area 0.016770367309243785 league 0.01673649619502486 party 0.016170796258541044 public 0.014879499758387406 action 0.014462554451976379
Topic #1: said 0.03931698353158134 did 0.020921712924128706 day 0.019104601609400414 time 0.01831709070799909 went 0.016737312124992396 didn 0.01578168091128047 came 0.015309992993991158 home 0.014066272404117154 years 0.013977943597090526 children 0.013018708896183822 started 0.012955370174476512 took 0.012889582364442695 saw 0.012757253461065704 told 0.012694070883029047 days 0.012505220162372863 just 0.012432963524113723 lord 0.012248382622198668 lef